In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
This carrys on the work from pre_process_articles.py that does the 
word embeeding. Now we create an list that stores the information of 
every articles in terms of their mapping from vectors to labels.
"""

def label_and_classification():
    import gensim
    import numpy as np
    import tensorflow as tf
    
    word_vector=gensim.models.KeyedVectors.load_word2vec_format('./pre_trained_wv/wv.txt')
    token_list=np.load('./pre_processed_data/token_list.npy',allow_pickle=True)
    token_list=token_list.tolist()
    
    """Two list below contain the words' vectors of X and the one-hot encoding labels of Y"""
    X,Y=[],[]
    def see_if_token_in_label_list(token,article_label):     
        lst=[w[1] for w in article_label]
        try:
            frt_occurence=lst.index(token[1])
        except ValueError:
            Y.append([0 if i !=18 else 1 for i in range(19)])
            return 18
        Y.append([0 if i !=label_mapping[article_label[frt_occurence][4]] else 1 for i in range(19)])
        return label_mapping[article_label[frt_occurence][4]]
    
    label_list=[]
    label_mapping={'name':0,'location':1,'time':2,'contact':3,'ID':4,'profession':5,'biomarker':6,
               'family':7,'clinical_event':8,'special_skills':9,'unique_treatment':10,'account':11,
               'organization':12,'education':13,'money':14,'belonging_mark':15,'med_exam':16,'others':17,'useless':18}
    
    for article in token_list:
        for l in article[1]:
            for i in range(3):
                l[i]=int(l[i])
                
        for token in article[0]:
            label_list.append(see_if_token_in_label_list(token,article[1])) 
            try:
                X.append(word_vector.get_vector(token[0]))
            except TypeError:
                X.append(np.zeros(300))
    c=0
    for v in X:
        if v[0]==0:
            c+=1
                
    '''Below tries to build an classifier for final classification'''
    number_of_split_train=int(len(X)*0.8)   
    train_X,train_Y,test_X,test_Y=X[:number_of_split_train],Y[:number_of_split_train],X[number_of_split_train:],Y[number_of_split_train:] #for banary classifier
    
    '''
    Since the label classes are extremely imbalanced, 
    we adjust the weights of each class by passing in the weight list to the model.
    '''
    number_of_label_class=[0 for _ in range(19)]
    for i in label_list:
        number_of_label_class[i]+=1
    weights_class_dict={i:len(label_list)/number_of_label_class[i] if number_of_label_class[i]!=0 else len(label_list) for i in range(19)}        
    
    """adjust the weight of 'useless' class so that we don't value this class too much."""
    weights_class_dict[18]=30 
# =============================================================================
#     METRICS = [
#           tf.keras.metrics.TruePositives(name='tp'),
#           tf.keras.metrics.FalsePositives(name='fp'),
#           tf.keras.metrics.TrueNegatives(name='tn'),
#           tf.keras.metrics.FalseNegatives(name='fn'), 
#           tf.keras.metrics.BinaryAccuracy(name='accuracy'),
#           tf.keras.metrics.Precision(name='precision'),
#           tf.keras.metrics.Recall(name='recall'),
#           tf.keras.metrics.AUC(name='auc')]
# =============================================================================

    model=tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(30,activation=tf.nn.relu,input_shape=(300,)))
    model.add(tf.keras.layers.Dense(15,activation=tf.nn.relu))
    model.add(tf.keras.layers.Dense(15,activation=tf.nn.relu))
    model.add(tf.keras.layers.Dense(15,activation=tf.nn.relu))
    model.add(tf.keras.layers.Dense(19,activation=tf.nn.softmax))
    model.compile(loss='CategoricalCrossentropy',optimizer='rmsprop',metrics=['accuracy'])
    model.summary()
    
    EPOCHS,BATCH_SIZE=2,10
    model.fit(np.array(train_X),np.array(train_Y),epochs=EPOCHS,batch_size=BATCH_SIZE,class_weight=weights_class_dict)
    test_loss,accuracy=model.evaluate(np.array(test_X),np.array(test_Y))
    predictions=model.predict(np.array(X))
    class_preds = np.argmax(predictions, axis=-1)
    
    """see what words are not in 'useless' category"""
    lst=set()
    for article,predict in zip(token_list,class_preds):
        for token in article[0]:
            if predict!=18:
                lst.add((token[0],predict))
    return lst


if __name__=='__main__':
    lst=label_and_classification()
    

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 30)                9030      
_________________________________________________________________
dense_1 (Dense)              (None, 15)                465       
_________________________________________________________________
dense_2 (Dense)              (None, 15)                240       
_________________________________________________________________
dense_3 (Dense)              (None, 15)                240       
_________________________________________________________________
dense_4 (Dense)              (None, 19)                304       
Total params: 10,279
Trainable params: 10,279
Non-trainable params: 0
_________________________________________________________________
Epoch 1/2
3740/3740 [==============================] - 9s 2ms/step - loss: 68.7037 - accuracy: 0.9515
Epoch 2/2
293/29

In [7]:
"""
Below are the words that are categorized tokens with labels and they don't make any sense at all, 
it's more of a random guess at best.
"""
for i in lst:
    print(i,end='')

('藥有', 2)('剛結束', 1)('預期', 1)('那種', 1)('這件', 1)('一次', 2)('預感', 1)('顏色會', 2)('一顆', 2)('變化', 2)('班輪', 1)('差別', 1)('上班', 1)('顆一顆', 1)('沒事', 1)('眼屎', 2)('伴侶關', 1)('看什麼樣', 1)('心情', 1)('還好', 2)('耶', 2)('上次', 2)('孫子', 2)('人比', 2)('火龍果', 2)('走', 2)('看看', 2)('嘿對', 2)('下降', 1)('分得', 1)('吃', 2)('嘖嘖嘖', 2)('元氣', 2)('然後也', 1)('泡', 2)('看到', 2)('選', 2)('癢', 2)('買', 2)('跡象', 2)('鼠蹊部', 2)('好好', 1)('抽血', 2)('需要', 2)('一直', 1)('決定', 1)('深', 2)('應該', 2)('一天', 1)('七千二', 2)('滿高', 1)('之下', 1)('黑', 2)('藥', 1)('性病', 1)('行為', 1)('這種', 2)('沒有', 2)('這陣子', 2)('白血球', 2)('OK', 1)('那一樣', 1)('淋巴', 2)('生活', 1)('然後四個', 1)('帶個', 2)('阿還', 2)('知道', 2)('56', 2)('老人家', 2)('目前', 2)('在家', 1)('那為', 2)('較屬', 1)('面覺', 1)('配合', 1)('人有', 2)('六班', 1)('容器', 2)('化痰', 2)('可能', 2)('膜炎', 2)('高風險', 1)('你現', 1)('24', 1)('要開', 2)('我覺', 2)('然後呢', 1)('感冒', 2)('一下', 2)('看電視', 2)('具有', 1)('抗生素', 2)('胯下', 2)('提出', 1)('然後無套', 1)('建議', 1)('係', 2)('回診', 2)('問題', 1)('情況', 1)('現在', 2)('三天', 2)('擔心', 2)('晚上', 1)('費是', 1)('那有', 1)('那天', 2)('禮拜四', 1)('前哨站'